In [2]:
import os, time
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
device = "cuda"

In [34]:
import torch.nn as nn
import torch.nn.functional as F
class convNet(nn.Module):
    def __init__(self,in_channels):
        super(convNet,self).__init__()
        self.c1 = nn.Conv2d(in_channels=in_channels, out_channels=64,kernel_size=3)
        self.bn1 = nn.BatchNorm2d(num_features=64,momentum=0.1)
        self.d1 = nn.Dropout(0.5)
        
        self.c2 = nn.Conv2d(64,64,3)
        self.bn2 = nn.BatchNorm2d(64, momentum=0.1)
        self.d2 = nn.Dropout(0.5)
        
        self.c3 = nn.Conv2d(64,128,3)
        self.bn3 = nn.BatchNorm2d(128, momentum=0.1)
        self.d3 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(128*3*3,256)
        self.fc2 = nn.Linear(256,256)
        self.fc3 = nn.Linear(256,256)
        self.out = nn.Linear(256,10)

    def forward(self,x):
        x = F.max_pool2d(self.c1(x), 2)
        x = F.relu(self.bn1(x))
        x = self.d1(x)
        
        x = F.max_pool2d(self.c2(x), 2)
        x = F.relu(self.bn2(x))
        x = self.d2(x)
        
        x = self.c3(x)
        x = F.relu(self.bn3(x))
        x = self.d3(x)
        
        x = x.view(-1, 128*3*3) #reshape
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.out(x)
                

In [ ]:
count_parameters(convNet(1))

In [5]:
def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            if param.dim() > 1:
                print(name, ':', 'x'.join(str(x) for x in list(param.size())), '=', num_param)
            else:
                print(name, ':', num_param)
            total_param += num_param
    return total_param

In [7]:
trans = transforms.Compose([
        transforms.RandomResizedCrop(28),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                              std=[0.229, 0.224, 0.225])
    ])

class KMnistDataset(Dataset):
    def __init__(self,len=None,transform=None):
        self.len = len
        self.transform = transform
        self.data = pd.read_csv("./dataset/train.csv")
        print("data shape:", np.shape(self.data))

    def __getitem__(self, idx):
        img = self.data.iloc[idx, 1:].values.astype(np.uint8).reshape((28, 28))
        label = self.data.iloc[idx, 0]  #(num, 1)
        img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)
            label = torch.as_tensor(label, dtype=torch.uint8)
        else:
            img = torch.as_tensor(img, dtype=torch.uint8)
            label = torch.as_tensor(label, dtype=torch.uint8)
            
        return img, label

    def __len__(self):
        return self.len if self.len!=None else len(self.imgs)

In [9]:
batch_size = 4
num_workers = 0
# kmnist_dataset = datasets.ImageFolder(kmnist_dataset, transform=None)
kmnist_dataset = KMnistDataset(len=20,transform=trans)
dataset_loader = DataLoader(kmnist_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

data shape: (60000, 785)


In [ ]:
kmnist_dataset.data.head(3)

In [ ]:
type(kmnist_dataset.data.iloc[20,1:].values)  #numpy ndarray
type(kmnist_dataset.data.iloc[20,0])  #numpy int64
kmnist_dataset.data.head(5)

In [ ]:
def main():
    epochs = 50000
    min_loss = 10000
    lr = 1e-3
    model = convNet(in_channels=1)
    model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=lr)
#     optimizer = torch.optim.SGD(net_Momentum.parameters(),lr=Learning_rate,momentum=0.8,nesterov=True)
#     optimizer = torch.optim.RMSprop(net_RMSprop.parameters(),lr=Learning_rate,alpha=0.9)
#     optimizer = torch.optim.Adam(model.parameters(),lr=lr,betas=(0.9,0.99))
#     optimizer = torch.optim.Adagrad(net_Adagrad.parameters(),lr=Learning_rate)
    
    for ep in range(1,epochs):
        model.train()
        for idx, data in enumerate(dataset_loader):
            img, target = data
            img, target = img.to(device), target.to(device,dtype=torch.long)
#             print(np.shape(img),np.shape(target)) #Tensor(4,1,28,28), Tensor(4)
#             print(np.max(img.cpu().numpy()),np.min(img.cpu().numpy())) #1.0 0.0
            pred = model(img)
#             print(pred.size())
#             print(target.size())
            loss = criterion(pred,target)
            if loss.item() < min_loss:
                print("===================Best Loss:==================")
                print(loss.item())
                min_loss = loss    
            if ep%100==0:
                print("Ep:{}, rounds:{}, loss:{}".format(ep,idx,loss))
                
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    ###Test
#     images, labels = data
#     outputs = net(images)
#     _, predicted = torch.max(outputs.data, 1)
#     total += labels.size(0)
#     correct += (predicted == labels).sum().item()

            
    
if __name__ == "__main__":
    main()
    

===================Best Loss:==================
2.3329193592071533
===================Best Loss:==================
2.2780697345733643
===================Best Loss:==================
2.277414321899414
===================Best Loss:==================
2.266037702560425
===================Best Loss:==================
2.26497745513916
===================Best Loss:==================
2.26129150390625
===================Best Loss:==================
2.259551525115967
Ep:100, rounds:0, loss:2.3284902572631836
Ep:100, rounds:1, loss:2.3189139366149902
Ep:100, rounds:2, loss:2.3024637699127197
Ep:100, rounds:3, loss:2.295393466949463
Ep:100, rounds:4, loss:2.300130605697632
===================Best Loss:==================
2.2537155151367188
Ep:200, rounds:0, loss:2.28549861907959
Ep:200, rounds:1, loss:2.320671558380127
Ep:200, rounds:2, loss:2.30633282661438
Ep:200, rounds:3, loss:2.286790370941162
Ep:200, rounds:4, loss:2.3055667877197266
Ep:300, rounds:0, loss:2.286898374557495
Ep:300, rounds:1, 